In [75]:
pip install  nbformat>=4.2.0

Note: you may need to restart the kernel to use updated packages.


In [76]:
pip freeze > ..//requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import numpy as np

## First, downloading data from the indicated source:

- [Dados da Vitivinicultura](http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_01) was indicated as main source of data;

In [40]:
def download(url = 'http://vitibrasil.cnpuv.embrapa.br/download/ExpVinho.csv', layer='raw'):
    import requests, os

    file_name = url.split('/')[-1:][0]
    file_path = f'..\data\{layer}\{file_name}'
    if not(os.path.isfile(file_path)):
        try:
            r = requests.get(url, allow_redirects=True)
            with open(file_path, 'wb') as file:
                file.write(r.content)
        except:
            raise
    return file_path, os.path.isfile(file_path)

urls_to_download = [
      'http://vitibrasil.cnpuv.embrapa.br/download/Comercio.csv'
    , 'http://vitibrasil.cnpuv.embrapa.br/download/ExpVinho.csv'
    , 'http://vitibrasil.cnpuv.embrapa.br/download/Producao.csv'

    ]
downloaded_files = {download(url = _url, layer='raw\\tech_challenge') for _url in urls_to_download}

In [71]:
for item in downloaded_files:
    print(item[1])

True
True
True


## Now, we need to search for climate & economic data from world:

- [WBPY](https://pypi.org/project/wbpy/) maybe can give us climate and economics data
- Was made from https://documents.worldbank.org/en/publication/documents-reports/api

In [60]:
import wbpy
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

api = wbpy.IndicatorAPI()

In [61]:
dict_countries = api.get_countries()
list_of_countries = list(dict_countries.keys())

http://api.worldbank.org/v2/country?format=json&per_page=10000


In [67]:
import time

dict_indicators = {
      'inflation_consumer_prices' : 'FP.CPI.TOTL.ZG'
    , 'gini_index' : 'SI.POV.GINI'
    , 'numberof_procedures_register_business' : 'IC.REG.PROC'
    , 'logistic_performance_index_1to5' : 'LP.LPI.OVRL.XQ'
    , 'domestic_credit_to_private_sector' : 'FS.AST.PRVT.GD.ZS'
    , 'population_growth' : 'SP.POP.GROW'
}

for key in dict_indicators:
    list_results = []
    for countries in chunker(list_of_countries, 10):
      time.sleep(5)
      try:
        df = api.get_dataset(dict_indicators[key], countries, date="1970:2021")
        list_results.append(pd.DataFrame(df.as_dict()))
      except:
        print(f"error in {countries}")
    pd.concat(list_results).to_csv(f"..\\data\\raw\\wbpy\\{key}.csv")

http://api.worldbank.org/v2/countries/ABW;ZH;AFG;A9;ZI;AGO;ALB;AND;1A;ARE/indicators/FP.CPI.TOTL.ZG?date=1970%3A2021&format=json&per_page=10000
http://api.worldbank.org/v2/countries/ARG;ARM;ASM;ATG;AUS;AUT;AZE;BDI;B4;B7/indicators/FP.CPI.TOTL.ZG?date=1970%3A2021&format=json&per_page=10000
http://api.worldbank.org/v2/countries/BEL;BEN;BFA;BGD;BGR;B1;BHR;BHS;BIH;B2/indicators/FP.CPI.TOTL.ZG?date=1970%3A2021&format=json&per_page=10000
http://api.worldbank.org/v2/countries/BLR;BLZ;B3;BMU;BOL;BRA;BRB;BRN;B6;BTN/indicators/FP.CPI.TOTL.ZG?date=1970%3A2021&format=json&per_page=10000
http://api.worldbank.org/v2/countries/BWA;C9;CAF;CAN;C4;B8;C5;CHE;JG;CHL/indicators/FP.CPI.TOTL.ZG?date=1970%3A2021&format=json&per_page=10000
http://api.worldbank.org/v2/countries/CHN;CIV;C6;C7;CMR;COD;COG;COL;COM;CPV/indicators/FP.CPI.TOTL.ZG?date=1970%3A2021&format=json&per_page=10000
http://api.worldbank.org/v2/countries/CRI;C8;S3;CUB;CUW;CYM;CYP;CZE;D4;D7/indicators/FP.CPI.TOTL.ZG?date=1970%3A2021&format=json&

OSError: Cannot save file into a non-existent directory: '..\data\raw\wbpy'

In [37]:
import os

os.environ['KAGGLE_USERNAME'] = "gpehls"  #manually input My_Kaggle User_Name 
os.environ['KAGGLE_KEY'] = "5a7b49ccb42f7975d9ea5203f455a770"  #

import kaggle
kaggle.api.authenticate()

# https://www.kaggle.com/datasets/sevgisarac/temperature-change
kaggle.api.dataset_download_files('sevgisarac/temperature-change', path='..\\data\\raw\\temp_change', unzip=True)

# https://www.kaggle.com/datasets/noaa/noaa-global-historical-climatology-network-daily
kaggle.api.dataset_download_files('noaa/noaa-global-historical-climatology-network-daily', path='..\\data\\raw\\noaa_global', unzip=True)